# Exploring the CommaAI Device data

This requires python 3.11 or later because of the usage of the tools folder from Openpilot, which uses 3.11 features

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import pandas as pd
from pathlib import Path
# This is taken from the Openpilot project
from openpilot.tools.lib.logreader import LogReader
from dataclasses import dataclass
from typing import List

/tmp/ipykernel_3072533/1313606866.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/ulrikro


In [4]:
device_path = Path("/home/ulrikro/datasets/CommaAI/2024-01-14--13-01-26--10/")

# About the folder

Each folder, marked by their data and a number, contains 5 files: ecamera.hevc, fcamera.hevc, qcamera.ts, qlog, rlog. Each file contains 1 min video. 

* ecamera.hevc - road facing wide angle camera 1928*1208 at 20Hz
* fcamera.hevc - road facing narrow camera 1928*1208 at 20Hz
* qcamera.ts - driver facing wide angle camera 526*330 at 20Hz
* qlog - some
* rlog - some



In [5]:
ecamera_path = device_path / "ecamera.hevc"
fcamera_path = device_path / "fcamera.hevc"
qcamera_path = device_path / "qcamera.ts"
qlog_path = device_path / "qlog"
rlog_path = device_path / "rlog"


# Overview of the video files

In [6]:
def read_video_frames(file_path: str):
    frames = []
    video = cv2.VideoCapture(file_path)
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        frames.append(frame)
    video.release()
    return frames

In [7]:
ecamera_frames = read_video_frames(ecamera_path.as_posix())
len(ecamera_frames)

1200

In [8]:
ecamera_frames[0].shape

(1208, 1928, 3)

In [9]:
fcamera_frames = read_video_frames(fcamera_path.as_posix())
len(fcamera_frames)

1200

In [10]:
fcamera_frames[0].shape

(1208, 1928, 3)

In [11]:
qcamera_frames = read_video_frames(qcamera_path.as_posix())
len(qcamera_frames)

1200

In [12]:
qcamera_frames[0].shape

(330, 526, 3)

# Overview of the log files

In [13]:
qlog_data = LogReader(qlog_path.as_posix())

In [14]:
qlog_list = list(qlog_data)
len(qlog_list)

11515

Each iteration throug the log data retruns a capnp.lib.capnp._DynamicStructReader object. 

capnp is a proto schema parsing library 

The proto schema seems to be defined in openpilot/common/params.cc
 


In [15]:
@dataclass
class CommandEntry:
    key: str
    value: str

@dataclass
class Commands:
    entries: List[CommandEntry]

@dataclass
class ParamEntry:
    key: str
    value: str

@dataclass
class Params:
    entries: List[ParamEntry]

@dataclass
class InitData:
    kernelArgs: List[str]
    dongleId: str
    deviceType: str
    version: str
    dirty: bool
    gitCommit: str
    gitBranch: str
    passive: bool
    gitRemote: str
    kernelVersion: str
    params: Params
    osVersion: str
    commands: Commands

@dataclass
class LogData:
    logMonoTime: int
    initData: InitData
    valid: bool
    # these just include a subset of the data

In [16]:
qlog_list: List[LogData] = list(qlog_data)

In [17]:
print([attr for attr in dir(qlog_list[0]) if not attr.startswith("__")])

['_get', '_get_by_field', '_has', '_has_by_field', '_parent', '_which', '_which_str', 'accelerometer', 'accelerometer2', 'androidGnssDEPRECATED', 'androidLog', 'applanixLocationDEPRECATED', 'applanixRawDEPRECATED', 'as_builder', 'boot', 'cameraOdometry', 'can', 'carControl', 'carParams', 'carState', 'cellInfoDEPRECATED', 'clocks', 'controlsState', 'customReserved0', 'customReserved1', 'customReserved2', 'customReserved3', 'customReserved4', 'customReserved5', 'customReserved6', 'customReserved7', 'customReserved8', 'customReserved9', 'customReservedRawData0', 'customReservedRawData1', 'customReservedRawData2', 'deviceState', 'driverCameraState', 'driverEncodeData', 'driverEncodeIdx', 'driverMonitoringState', 'driverStateDEPRECATED', 'driverStateV2', 'errorLogMessage', 'ethernetDataDEPRECATED', 'featuresDEPRECATED', 'gnssMeasurements', 'gpsLocation', 'gpsLocationExternal', 'gpsNMEA', 'gpsPlannerPlanDEPRECATED', 'gpsPlannerPointsDEPRECATED', 'gyroscope', 'gyroscope2', 'initData', 'is_roo

In [18]:
print(qlog_list[7])

( logMonoTime = 1151024117684,
  carState = (
    vEgo = 11.12378,
    gas = 0,
    gasPressed = false,
    brake = 0,
    brakePressed = false,
    steeringAngleDeg = 0,
    steeringTorque = 0,
    steeringPressed = false,
    cruiseState = (
      enabled = false,
      speed = 0,
      available = false,
      speedOffset = 0,
      standstill = false,
      nonAdaptive = false,
      speedCluster = 0 ),
    events = [
      ( name = carUnrecognized,
        enable = false,
        noEntry = false,
        warning = false,
        userDisable = false,
        softDisable = false,
        immediateDisable = false,
        preEnable = false,
        permanent = true,
        overrideLongitudinal = false,
        overrideLateral = false ) ],
    gearShifter = unknown,
    steeringRateDeg = 0,
    aEgo = 0,
    vEgoRaw = 11.12378,
    standstill = false,
    brakeLightsDEPRECATED = false,
    leftBlinker = false,
    rightBlinker = false,
    yawRate = 0,
    genericToggle = false,
    

In [19]:
def isiterable(item):
    try:
        iter(item)
        return True
    except TypeError:
        return False

def get_fields(o):
    if isinstance(o, list) or isinstance(o, tuple) or isinstance(o, set) or isiterable(o):
        o = list(o)
        total_attrs = set()
        example_obj = dict()
        for item in o:
            item_attrs = [attr for attr in dir(item) if not attr.startswith("__")]
            total_attrs.update(item_attrs)
            for attr in item_attrs:
                if attr not in example_obj:
                    example_obj[attr] = item
        
        type_dict = dict()
        for attr in total_attrs:
            type_dict[attr] = get_fields(example_obj[attr])
        return type_dict
    elif isinstance(o, dict):
        total_dict = dict()
        for key, value in o.items():
            total_dict[key] = get_fields(value)
        return total_dict
    elif isinstance(o, str) or isinstance(o, int) or isinstance(o, float):
        return type(o).__name__
    else:
        item_attrs = [attr for attr in dir(o) if not attr.startswith("__")]
        total_dict = dict()
        for attr in item_attrs:
            total_dict[attr] = get_fields(getattr(o, attr))
        
        

In [20]:
get_fields(qlog_list)

KjException: capnp/dynamic.c++:141: failed: expected isSetInUnion(field); Tried to get() a union member which is not currently initialized.; field.getProto().getName() = accelerometer; schema.getProto().getDisplayName() = log.capnp:Event
stack: 7f3c8c69b389 7f3c8c69eafa 7f3c8c69f8df 7f3c8c562a3d 7f3c8c52fecd 7f3cddc6d12a 7f3cddbbc3dd 7f3cddd113ae 7f3cddd0d27f 7f3cddbbc51b 7f3cddc30374 7f3cddbc05b7 7f3cddc30374 7f3cddbc05b7 7f3cddc31454 7f3cddbbc885 7f3cddd11510 7f3cddc1a0b7 7f3cddc16cef 7f3cddbb9eac 7f3cddc30374 7f3cddbc05b7 7f3cddc30374 7f3cddbc05b7 7f3cddc30374 7f3cddbc05b7 7f3cddc30374 7f3cddbc05b7 7f3cddc30374 7f3cddbc05b7 7f3cddc30374